In [1]:
import pandas as pd
import numpy as np
import pickle
from transformers import AutoTokenizer, AutoModel
from datasets import Dataset
from datasets import load_dataset
import datasets
import torch
from collections import Counter

# import string library function  
import string

c:\Users\halom\anaconda3\envs\test\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [67]:
#This notebook was used to fix our tf-idf implementation.
# Made it vectorized, we're good!

In [2]:
dataset_dict = datasets.load_from_disk("./Dataset/CodeSearchCorpus/")
train_dataset = dataset_dict["train"]
np.random.seed(1)
train_subset_indices = np.random.choice(len(train_dataset), 10000, replace = False)
train_dataset_subset = train_dataset.select(train_subset_indices)


In [3]:
# train_dataset_subset[0]["func_code_tokens"]
tsed_DF = train_dataset_subset.to_pandas() #train-subset-embeddings-dataset_DF
def clean_code_tokens(lst):
    result = string.punctuation 
    new_lst = [] 
    for character in lst:
        if character in result:
            continue
        else:
            new_lst.append(character)
    return new_lst

tsed_DF["clean_code_tokens"] =  tsed_DF["func_code_tokens"].apply(clean_code_tokens)


In [31]:
documents = tsed_DF["clean_code_tokens"].to_dict()

all_words = []
for i in list(tsed_DF["clean_code_tokens"].to_dict().values()):
    all_words += i

all_words = list(set(all_words))
all_words


try:
     with open('./pickleObjects/inverted_index.pkl', 'rb') as f:
        inverted_index = pickle.load(f) # deserialize using load()
        f.close()
except:
    inverted_index = {}

    for word in all_words:
            if word != "":
                lst_docs = []
                for i, doc in documents.items():
                    if word in doc:
                        lst_docs.append(i)
            
                inverted_index[word] = lst_docs

In [32]:
len(all_words) == len(inverted_index)

True

In [54]:
def doc_freq(word):
    c = 0
    try:
        c = inverted_index[word]
    except:
        pass

    if type(c) == list:
        return len(c)
    else:
        return 0



N = 10000 #number of rows sampled
tf_idf = {}
for i in range(N):
    # print(i)
    tokens = tsed_DF["clean_code_tokens"].iloc[i]
    counter = Counter(tokens)
    words_count = len(tokens)

    for token in np.unique(tokens):
        tf = counter[token] / words_count
        df = doc_freq(token)
        idf = np.log((N + 1) / (df + 1))

        tf_idf[i, token] = tf * idf
    

In [55]:
tsed_DF["clean_code_tokens"].iloc[1]

['static',
 'void',
 'appendDot',
 'Node',
 'n',
 'ControlFlowGraph',
 'Node',
 'inCFG',
 'Appendable',
 'builder',
 'throws',
 'IOException',
 'new',
 'DotFormatter',
 'n',
 'inCFG',
 'builder',
 'false']

In [56]:
# inverted_index
tf_idf

{(0, "'rb'"): 0.3406272547089073,
 (0, "'utf-8'"): 0.2888226581245685,
 (0, 'as'): 0.12951835802230802,
 (0, 'config'): 0.3657480782054899,
 (0, 'decode'): 0.26295983241975446,
 (0, 'def'): 0.0701668538425268,
 (0, 'fi'): 0.6812545094178146,
 (0, 'file_bytes'): 0.8517293186416572,
 (0, 'load'): 0.23780465353615043,
 (0, 'load_config'): 0.4258646593208286,
 (0, 'open'): 0.23178646942365666,
 (0, 'path'): 0.3156001020707146,
 (0, 'read'): 0.23076602584209632,
 (0, 'return'): 0.013320602085796086,
 (0, 'with'): 0.22114743120972352,
 (0, 'yaml'): 0.34539276369912353,
 (1, 'Appendable'): 0.4506571154615781,
 (1, 'ControlFlowGraph'): 0.4731829548009206,
 (1, 'DotFormatter'): 0.4731829548009206,
 (1, 'IOException'): 0.2062928967601841,
 (1, 'Node'): 0.5648117757701582,
 (1, 'appendDot'): 0.4731829548009206,
 (1, 'builder'): 0.5595614552312088,
 (1, 'false'): 0.12308835450796916,
 (1, 'inCFG'): 0.9463659096018412,
 (1, 'n'): 0.46444471617600896,
 (1, 'new'): 0.08409851585833636,
 (1, 'static')

In [57]:
len(tf_idf)

275120

In [58]:
all_words
all_words_dict = dict(zip(all_words, range(len(all_words))))

In [59]:
all_words_dict

{'enableQueryLogging': 0,
 'default_search_depth': 1,
 'memoryCopyTest': 2,
 '"The VPCS relay feature could not be disabled because the VPCS version is below 0.8b"': 3,
 "'pidfile'": 4,
 "'CREATE OR REPLACE FUNCTION check_javascript_function(js text) RETURNS boolean LANGUAGE plv8 VOLATILE AS '": 5,
 'AbstractJaxb': 6,
 'optimizeDataSource': 7,
 "'Merlin Park Hospital Merlin Park'": 8,
 'stamp': 9,
 'projection': 10,
 'ScriptHandler': 11,
 'UserRecord': 12,
 "'displayofgrade'": 13,
 'aCompletedOrderStepList': 14,
 'getPasswordHtml': 15,
 'reporter': 16,
 "'Letterkenny Shopping Centre'": 17,
 'Vector3dc': 18,
 'ClearExistingRulesEqual': 19,
 'dataDgst': 20,
 'getDataModels': 21,
 'HEADER_LEVEL_INDEX': 22,
 'WatchModelMachines': 23,
 '# evaluating string for support of nested lists and other complex data structures': 24,
 '//    System.out.println("jo!");': 25,
 'possibleOldHash': 26,
 '"BYYEARDAY"': 27,
 'authList': 28,
 '// . .. : :: , ;\r': 29,
 'cusolverSpHandle': 30,
 'InputRegister'

In [60]:
#Vectorising tf-idf

D = np.zeros((N, len(all_words)))

for i in tf_idf:
    try:
        ind = all_words_dict[i[1]]
        D[i[0]][ind] = tf_idf[i]
    except:
        pass

In [61]:
D[2].sum()

6.346189832260017

In [63]:
np.zeros((N, len(all_words))).shape

(10000, 98114)

In [64]:
def gen_vector(tokens):

    Q = np.zeros((len(all_words)))
    
    counter = Counter(tokens)
    words_count = len(tokens)

    query_weights = {}
    
    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log((N+1)/(df+1))

        try:
            ind = all_words_dict[token]
            Q[ind] = tf*idf
        except:
            pass
    return Q

In [66]:
gen_vector("pandas how to select first 10 rows".split()).sum()

3.7283742309983863